![](../assets/images/construction.gif)

# Journey 4
## Part 1


### Transfer Learning 



![Apple CNN](./images/AppleCNN-1.png)

In [ ]:
from os import listdir
from os.path import isdir, isfile, join
import re
import random
import re
import os
import sys

import numpy as np
from PIL import Image

In [ ]:
DATA_DIR = '../data/fruits-360-data/fruits-360'
TEST_DATA = DATA_DIR + '/Test'
TRAIN_DATA = DATA_DIR + '/Training'

In [ ]:
#
# Create a one-hot encoding for the given list
#
def create_one_hot_encoding_dicts(list_to_encode):
    list_to_one_hot = dict()
    one_hot_to_item = dict()
    for i, l in enumerate(list_to_encode):
        oh = np.zeros((len(list_to_encode)))
        oh[i] = 1
        list_to_one_hot[l] = oh
        one_hot_to_item[np.array2string(oh)] = l
    return list_to_one_hot, one_hot_to_item

In [ ]:
#
# Return a list of only the JPG files for a given dir
#
def list_files(data_dir):
    jpg_files = [f for f in listdir(data_dir) if isfile(join(data_dir, f)) and re.search('\.jpg$',f,flags=re.IGNORECASE)]
    return jpg_files

In [ ]:
#
# Load a given list of fruits from the selected data set location
#
def load_data(fruits_to_get, 
              test_data_dir,
              train_data_dir
             ):
    
    one_hot_dict, item_dict = create_one_hot_encoding_dicts(fruits_to_get)
        
    test_data = []
    train_data = []
    
    for fruit in fruits_to_get:
        
        test_fruit_dir = test_data_dir + "/" + fruit
        train_fruit_dir = train_data_dir + "/" + fruit
        
        if isdir(test_fruit_dir) and isdir(train_fruit_dir):
            one_hot_val = one_hot_dict[fruit]
            
            print("loading [" + fruit + "]" + " one hot: " + str(one_hot_val))
            
            print('Test Data')
            for img_file in list_files(test_fruit_dir):
                test_data.append([np.array(Image.open(join(test_fruit_dir,img_file))), one_hot_val])
                print('.', end='')
            print('')

            print('Training Data')
            for img_file in list_files(train_fruit_dir):
                train_data.append([np.array(Image.open(join(train_fruit_dir,img_file))), one_hot_val])
                print('.', end='')
            print('')

            img_shape = test_data[0][0].shape
            one_hot_shape = test_data[0][1].shape
            
            # Convert data to numpy
            x_test = np.zeros((len(test_data), *img_shape))
            y_test = np.zeros((len(test_data), one_hot_shape[0]))
            x_train = np.zeros((len(train_data), *img_shape))
            y_train = np.zeros((len(train_data), one_hot_shape[0]))
            
            i = 0 
            rnd_idx = random.sample(range(0, len(test_data)), len(test_data))
            for img, one_hot in test_data:
                x_test[rnd_idx[i]] = img / 255.0  # rescale 0.0 to 1.0
                y_test[rnd_idx[i]] = one_hot
                i += 1

            i = 0 
            rnd_idx = random.sample(range(0, len(train_data)), len(train_data))
            for img, one_hot in train_data:
                x_train[rnd_idx[i]] = img / 255.0  # rescale 0.0 to 1.0
                y_train[rnd_idx[i]] = one_hot
                i += 1

        else:
            raise Exception("Cannot load unknown fruit - missing from test and/or train data set[" + fruit + "]")
            
    return x_train, \
           y_train, \
           x_test, \
           y_test, \
           one_hot_dict, \
           item_dict
        

In [ ]:
list_to_get = ["Apple Golden 1",
               "Apple Golden 2",
               "Apple Golden 3",
               "Apple Red 1",
               "Apple Red 2",
               "Apple Red 3"]              
x_train, y_train, x_test, y_test, _, item_dict = load_data(list_to_get, TEST_DATA, TRAIN_DATA)

In [ ]:
print(x_test.shape)
print(y_test.shape)
print(x_train.shape)
print(y_train.shape)
        

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline  

for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(x_train[i], interpolation='none')

In [ ]:
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.datasets import mnist
from keras.layers import Input, Dense, Conv2D, Flatten, MaxPooling2D, BatchNormalization, Dropout
from keras.losses import categorical_crossentropy
from keras.models import Model, Sequential
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.utils import multi_gpu_model
import tensorflow as tf

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
def build_cnn_layers(model_name,
                     input_shape):
    cnn_in = Input(shape=input_shape, name = "cnn0Input")
    cnn_l1 = Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='relu', name = "cnn1_Conv2d")(cnn_in) 
    cnn_l2 = MaxPooling2D(pool_size=(2, 2), name = "cnn2_MaxPool2D")(cnn_l1)
    cnn_l3 = BatchNormalization(name = "cnn3_BatchNorm")(cnn_l2) 
    cnn_l4 = Conv2D(32, kernel_size=(3, 3), strides=(2, 2), activation='relu', name = "cnn3_conv2d")(cnn_l3)
    cnn_l5 = MaxPooling2D(pool_size=(2, 2), name = "cnn5_MaxPool2D")(cnn_l4)
    cnn_l6 = BatchNormalization(name = "cnn6_BatchNorm")(cnn_l5) 
    cnn_l7 = Dropout(rate=0.25,name = "cnn7_Dropout")(cnn_l6) 
    cnn_out = Flatten(name = "cnnOutput_Flatten")(cnn_l7)
    return cnn_in, cnn_out

In [ ]:
def build_cnn_layers(model_name,
                     input_shape):
    cnn_in = Input(shape=input_shape, name = "cnn0Input")
    cnn_l1 = Conv2D(4, kernel_size=(5, 5), strides=(2, 2), activation='relu', name = "cnn1_Conv2d")(cnn_in) 
    cnn_l2 = MaxPooling2D(pool_size=(2, 2), name = "cnn2_MaxPool2D")(cnn_l1)
    cnn_l3 = BatchNormalization(name = "cnn3_BatchNorm")(cnn_l2) 
    cnn_l7 = Dropout(rate=0.25,name = "cnn7_Dropout")(cnn_l3) 
    cnn_out = Flatten(name = "cnnOutput_Flatten")(cnn_l7)
    return cnn_in, cnn_out

In [ ]:
def build_classifier_layers(model_name,
                            num_classes,
                            input_layer):
    cl_l1 = Dense(25, activation='relu', name = "clInput_Dense")(input_layer) # 500 -> 50
    cl_l2 = Dropout(rate=0.25, name = "cl1_Dropout")(cl_l1)
    cl_out = Dense(num_classes, activation='softmax', name = "clOutput")(cl_l2)
    return cl_out

In [ ]:
def create_model(model_name,
                input_shape,
                num_classes):
    model_cnn_in, model_cnn_out = build_cnn_layers(model_name, input_shape)
    model_cl_out = build_classifier_layers(model_name, num_classes, model_cnn_out)

    model = Model(inputs=[model_cnn_in],outputs=[model_cl_out])
    model.name = model_name
    return model

In [ ]:
input_shape = (100, 100, 3)
num_classes = 6
model_name = "classifier_" + str(num_classes)

model = create_model(model_name, input_shape, num_classes)

print(model.summary())

In [ ]:
def model_compile(model):
    model = multi_gpu_model(model, gpus=2)
    model.compile(loss=categorical_crossentropy,
                  optimizer=SGD(lr=0.01),
                  metrics=['accuracy'])
    return model

In [ ]:
model = model_compile(model)

In [ ]:
batch_size = 32
num_epochs = 20
history = model.fit(x_train, y_train,
                    epochs=num_epochs,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(x_test, y_test)
                    )

In [ ]:
def plot_results(history):
    color_r = 'tab:red'
    color_b = 'tab:blue'

    fig, _ = plt.subplots(figsize=(15,6))
    
    # Loss
    ax1 = plt.subplot(1,2,1)
    plt.title('Loss')
    ax1.set_xlabel('Epoc')
    ax1.set_ylabel('Traing Loss', color=color_r)
    ax1.plot(epocs, history.history['loss'], color=color_r)
    ax1.tick_params(axis='y', labelcolor=color)
    ax2 = ax1.twinx()  # 2nd Axis for Validation Loss
    ax2.set_ylabel('Validation Loss', color=color_b)  
    ax2.plot(epocs, history.history['val_loss'], color=color_b)
    ax2.tick_params(axis='y', labelcolor=color)

    # Accuracy
    ax1 = plt.subplot(1,2,2)
    plt.title('Accuracy')
    ax1.set_xlabel('Epoc')
    ax1.set_ylabel('Traing accuracy', color=color_r)
    ax1.plot(epocs, history.history['acc'], color=color_r)
    ax1.tick_params(axis='y', labelcolor=color)
    ax2 = ax1.twinx()  # 2nd Axis for Validation Loss
    ax2.set_ylabel('Validation Accuracy', color=color_b)  
    ax2.plot(epocs, history.history['val_acc'], color=color_b)
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.show()
    return

In [ ]:
plot_results(history)

In [ ]:
def test_model_prediction(model,
                          x_test_set, 
                          y_test_set,
                          num_to_test,
                          summary = True):
    num_passed = 0
    list_to_test = np.random.randint(len(x_test_set), size=num_to_test)

    for i in list_to_test:
        xt = x_test_set[i]
        xt = np.reshape(xt,(1,100,100,3))
        pred = model.predict(xt)
        if(np.argmax(pred) == np.argmax(y_test_set[i])):
            if not summary:
                print("Correct Prediction for x_test element: " + str(i)) 
            num_passed += 1
        else:
            if not summary:
                print("Failed Prediction for x_test element: " + str(i))
    print("\nOverall Score : "+ str(round(num_passed/num_to_test,1)*100)+"%")
    return

In [ ]:
test_model_prediction(model, x_test, y_test, 9, summary=False)

In [ ]:
list_to_get = ["Apple Braeburn"]              
x1_train, y1_train, x1_test, y1_test, _, item_dict = load_data(list_to_get, TEST_DATA, TRAIN_DATA)

In [ ]:
# 

In [ ]:
test_model_prediction(model, x1_test, y1_test, 10)

In [ ]:
cnn_weights = model.get_weights()

In [ ]:
for i, w in enumerate(cnn_weights):
    print(str(i) + ' : ' + str(w.shape))

In [ ]:
def model_layers(model):
    try:
        layers = model.get_layer(model_name).layers
    except:
        layers = model.layers
        
    return layers

In [ ]:
# Model strcuture changes when complied for GPU, so we need to look in diffent places for the 'actual' layers
def aij_model_summary(model):
    
    for l in model_layers(model):
        print(l.name+":: params #"+str(l.count_params()))
    return

In [ ]:
aij_model_summary(model)

## Articles of Interest

[gpu article 1](https://datascience.stackexchange.com/questions/23895/multi-gpu-in-keras)
[Keras Conv2D](https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/)

In [ ]:
# Model strcuture changes when complied for GPU, so we need to look in diffent places for the 'actual' layers
cnn_only = ".*cnn.*"
def aij_save_cnn_layers(model,
                        pattern=".*"):

    for l in model_layers(model):
        l_name = l.name
        if re.search(pattern, l_name):
            w = l.get_weights()
            print('Saving Layer :'+ l.name + ":: params #"+str(l.count_params()))
            np.save(l_name + '.npy', w, allow_pickle=True)
    return

In [ ]:
aij_save_cnn_layers(model, pattern= ".*")

In [ ]:
# Model strcuture changes when complied for GPU, so we need to look in diffent places for the 'actual' layers
def aij_load_cnn_layers(model,
                        pattern = ".*",
                        trainable = True):
    
    for l in model_layers(model):
        try:
            if re.search(pattern, l.name):
                w = np.load(l.name+'.npy',allow_pickle=True)
                l.set_weights(w)
                l.trainable = trainable
                print("Loaded weights for Layer: " + l.name + 'Trainable: ' + str(trainable))
            else:
                print("Ignored Layer:" + l.name + " as it does not match load pattern")
        except IOError as e:
            print("No loadable weights for Layer: "+l.name) 
        except:
            print("Unexpected error:", sys.exc_info()[0])
    return

In [ ]:
# Test we can create a new 6 classifer model and re load
input_shape = (100, 100, 3)
num_classes = 6
model_name = "classifier_reload_test" + str(num_classes)

model_reload_test = create_model(model_name, input_shape, num_classes)
model_reload_test = model_compile(model_reload_test)
aij_model_summary(model_reload_test)


In [ ]:
test_model_prediction(model_reload_test, x_test, y_test, 10)

In [ ]:
aij_load_cnn_layers(model_reload_test)
model = model_compile(model)

In [ ]:
test_model_prediction(model_reload_test, x_test, y_test, 10)

In [ ]:
input_shape = (100, 100, 3)
num_classes = 7
model_name = "classifier_" + str(num_classes)

model_7_classes = create_model(model_name, input_shape, num_classes)

print(model_7_classes.summary())

In [ ]:
# Need to recompile model after load
aij_load_cnn_layers(model_7_classes, pattern = "cnn.*", trainable = False)


In [ ]:
model_7_classes = model_compile(model_7_classes)

In [ ]:
def one_hot_transform_6_to_7_class(y_6c):
    sh = y_6c.shape
    y_7c = np.zeros((sh[0], sh[1]+1))
    y_7c[:,:-1] = y_6c
    return y_7c

In [ ]:
y_train_7c = one_hot_transform_6_to_7_class(y_train)
y_test_7c = one_hot_transform_6_to_7_class(y_test)

In [ ]:
test_model_prediction(model_7_classes, x_test, y_test_7c, 10)

In [ ]:
batch_size = 32
num_epochs = 5
history = model_7_classes.fit(x_train, y_train_7c,
                              epochs=num_epochs,
                              batch_size=batch_size,
                              shuffle=True,
                              validation_data=(x_test, y_test_7c)
                             )

In [ ]:
test_model_prediction(model_7_classes, x_test, y_test_7c, 10)

In [ ]:
epocs = np.arange(0,num_epochs,1)

fig, ax1 = plt.subplots()
plt.title('Loss')

color = 'tab:red'
ax1.set_xlabel('Epoc')
ax1.set_ylabel('Traing Loss', color=color)
ax1.plot(epocs, history.history['loss'], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Validation Loss', color=color)  # we already handled the x-label with ax1
ax2.plot(epocs, history.history['val_loss'], color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
def one_hot_single_class_to_7_class(y_1c):
    sh = y_1c.shape
    y_7c = np.zeros((sh[0], sh[1]+6))
    y_7c[:,6] = y_1c.reshape((sh[0]))
    return y_7c

In [ ]:
y1_test_7c = one_hot_single_class_to_7_class(y1_test)
y1_train_7c = one_hot_single_class_to_7_class(y1_train)
print(y1_test_7c.shape)
print(y1_test_7c[0])

In [ ]:
test_model_prediction(model_7_classes, x1_test, y1_test_7c, 10)

In [ ]:
x_all_train = np.concatenate((x_train, x1_train), axis=0)
x_all_test = np.concatenate((x_test, x1_test), axis=0)
y_all_train = np.concatenate((y_train_7c, y1_train_7c), axis=0)
y_all_test = np.concatenate((y_test_7c, y1_test_7c), axis=0)
print(x_all_train.shape)
print(x_all_test.shape)
print(y_all_train.shape)
print(y_all_test.shape)

In [ ]:
batch_size = 32
num_epochs = 5
history = model_7_classes.fit(x_all_train, y_all_train,
                              epochs=num_epochs,
                              batch_size=batch_size,
                              shuffle=True,
                              validation_data=(x_all_test, y_all_test)
                             )

In [ ]:
test_model_prediction(model_7_classes, x_test, y_test_7c, 10)

In [ ]:
test_model_prediction(model_7_classes, x1_test, y1_test_7c, 10)